---
title: Building a GPT - companion notebook qmd
author: Andrey Karpathy
date: '2025-04-11'
freeze: true
jupyter: 
  kernelspec:
    name: "conda-env-gene46100-py"
    language: "python"
    display_name: "gene46100"
format:
  html:
    code-fold: true
    code-line-numbers: true
    code-tools: true
    code-wrap: true
description: Companion notebook from Karpathys video on building a minimal GPT, annotated by cursors LLM with summary from gemini.
---




## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT. Downloaded from [here](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing)

(https://github.com/karpathy/nanoGPT)

## Data Preparation

### Step 1: Loading & Inspection


In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
# print the length of the dataset
print("length of dataset in characters: ", len(text))

In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

### Step 2: Tokenization


In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

### Step 3: Creating Batches for Training


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

In [ ]:
# split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# define the block size (context length)
block_size = 8
train_data[:block_size+1]

### Understanding the Context and Target


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

### Creating Batches


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

## Model Architecture

### Starting Simple: Bigram Language Model
This model predicts the next character based on the current character only. 
It uses the logits for the next character in a lookup table.

Key points:
- Simplest possible model
- Uses an Embedding Table of size vocab_size x vocab_size
- Each row contains predicted scores for next character
- Ignores context beyond last character


In [ ]:
# define the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # Text Generation Method
    # This method implements autoregressive text generation by:
    # 1. Taking the current context (idx)
    # 2. Predicting the next token probabilities
    # 3. Sampling from these probabilities
    # 4. Appending the sampled token to the context
    # 5. Repeating for max_new_tokens iterations
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

### Understanding Cross Entropy Loss
Loss = -Σ(y * log(p))
where:
- y = actual probability (0 or 1)
- p = predicted probability
- Σ = sum over all classes

This is the loss for a single token. The total loss is the average across all B*T tokens in the batch, where:
- B = batch size (number of sequences processed in parallel)
- T = sequence length (number of tokens in each sequence)

Before training, we would expect the model to predict the next character from a uniform distribution.

-Σ(y * log(p)) = 
= - ( 1 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) ) = - log(1/65) 
= 4.1744 per token.

i.e. at the beginning we expect loss of -log(1/65) = 4.1744 per token.

### Model Initialization and Training


In [ ]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

### Text Generation


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

### Training Setup


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

### Training Loop


In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
# %pip install torch torchvision torchaudio scikit-learn

# import os
# import time
# import math
# import pickle
# from contextlib import nullcontext

# import numpy as np
# import torch
# from torch.nn.parallel import DistributedDataParallel as DDP
# from torch.distributed import init_process_group, destroy_process_group

# from model import GPTConfig, GPT

In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

In [ ]:
# split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# define the block size
block_size = 8
train_data[:block_size+1]

### define the context and target: 8 examples in one batch


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

### define the batch size and get the batch


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

### start with a simple model: the bigram language model
This model predicts the next character based on the current character only. 
It uses the logits for the next character in a lookup table.


In [ ]:
# define the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # Text Generation Method
    # This method implements autoregressive text generation by:
    # 1. Taking the current context (idx)
    # 2. Predicting the next token probabilities
    # 3. Sampling from these probabilities
    # 4. Appending the sampled token to the context
    # 5. Repeating for max_new_tokens iterations
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

### cross entropy loss
Loss = -Σ(y * log(p))
where:
- y = actual probability (0 or 1)
- p = predicted probability
- Σ = sum over all classes

This is the loss for a single token. The total loss is the average across all B*T tokens in the batch, where:
- B = batch size (number of sequences processed in parallel)
- T = sequence length (number of tokens in each sequence)

Before training, we would expect the model to predict the next character from a uniform distribution.

-Σ(y * log(p)) = 
= - ( 1 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) + 0 * log(1/65) ) = - log(1/65) 
= 4.1744 per token.

i.e. at the beginning we expect loss of -log(1/65) = 4.1744 per token.


### initialize the model and compute the loss


In [ ]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

### generate text


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

### choose AdamW as the optimizer


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

### train the model


In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

### generate text starting with \n as initial context


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

## Self-Attention: The Key Innovation

### Understanding Self-Attention
Self-attention allows tokens to communicate across the sequence in a data-dependent way. Each token can "look" at previous tokens and decide which ones are most relevant for predicting the next token.

### The Mathematical Trick: Weighted Aggregation


In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

### Version 1: Using a For Loop


In [ ]:
# consider the following toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

### Version 2: Matrix Multiplication


In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

### Version 3: Using Softmax


In [ ]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

### Version 4: Self-Attention


In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

### Why Scaled Attention?
- Dividing by sqrt(head_size) prevents scores from becoming too large
- Keeps softmax from producing overly sharp distributions
- Aids training stability


In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

In [ ]:
q.var()

In [ ]:
wei.var()

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

## Layer Normalization
Layer normalization normalizes features across the embedding dimension for each token independently, which helps stabilize training.


In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

### Full finished code, for reference


In [ ]:
# Import necessary PyTorch modules
import torch
import torch.nn as nn
from torch.nn import functional as F

# ===== HYPERPARAMETERS =====
# These are the key parameters that control the model's behavior and training
batch_size = 16        # Number of independent sequences processed in parallel
block_size = 32        # Maximum context length for predictions (like a window size)
max_iters = 5000       # Total number of training iterations
eval_interval = 100    # How often to evaluate the model during training
learning_rate = 1e-3   # Step size for gradient descent
eval_iters = 200       # Number of iterations to average over when evaluating
n_embd = 64           # Size of the embedding dimension (d_model in original paper)
n_head = 4            # Number of attention heads
n_layer = 4           # Number of transformer layers
dropout = 0.0         # Probability of dropping out neurons (0 = no dropout)
# ==========================

# Device selection: MPS (Apple Silicon) > CUDA > CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon GPU
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU
else:
    device = torch.device("cpu")   # CPU fallback
print(f"Using device: {device}")

# Set random seed for reproducibility
torch.manual_seed(1337)
if device.type == 'cuda':
    torch.cuda.manual_seed(1337)
elif device.type == 'mps':
    torch.mps.manual_seed(1337)

# Load and read the training text
# Note: This assumes 'input.txt' exists in the current directory
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# ===== DATA PREPROCESSING =====
# Create vocabulary from unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# Create mapping between characters and integers
stoi = { ch:i for i,ch in enumerate(chars) }  # string to index
itos = { i:ch for i,ch in enumerate(chars) }  # index to string
# Define encoder and decoder functions
encode = lambda s: [stoi[c] for c in s]  # convert string to list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # convert list of integers to string

# Split data into training and validation sets
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))  # first 90% for training
train_data = data[:n]
val_data = data[n:]
# =============================

# ===== DATA LOADING FUNCTION =====
def get_batch(split):
    """Generate a batch of data for training or validation.
    
    Args:
        split: 'train' or 'val' to specify which dataset to use
    
    Returns:
        x: input sequence of shape (batch_size, block_size)
        y: target sequence of shape (batch_size, block_size)
    """
    data = train_data if split == 'train' else val_data
    # Randomly select starting indices for sequences
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Create input and target sequences
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
# ================================

# ===== LOSS ESTIMATION FUNCTION =====
@torch.no_grad()  # Disable gradient calculation for efficiency
def estimate_loss():
    """Estimate the loss on training and validation sets.
    
    Returns:
        Dictionary containing average loss for train and validation sets
    """
    out = {}
    model.eval()  # Set model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()  # Set model back to training mode
    return out
# ===================================

# ===== ATTENTION HEAD IMPLEMENTATION =====
class Head(nn.Module):
    """Single head of self-attention."""
    
    def __init__(self, head_size):
        super().__init__()
        # Key, Query, Value projections
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # Create lower triangular mask for causal attention
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        # Compute key, query, and value matrices
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # Compute attention scores
        wei = q @ k.transpose(-2,-1) * C**-0.5  # Scale by sqrt(d_k)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # Apply mask
        wei = F.softmax(wei, dim=-1)  # Convert to probabilities
        wei = self.dropout(wei)
        # Weighted aggregation of values
        v = self.value(x)
        out = wei @ v
        return out
# ========================================

# ===== MULTI-HEAD ATTENTION =====
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel."""
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)  # Projection layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Process through each head and concatenate results
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
# ===============================

# ===== FEED-FORWARD NETWORK =====
class FeedFoward(nn.Module):
    """Simple feed-forward network with one hidden layer."""
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # Expand dimension
            nn.ReLU(),                      # Non-linearity
            nn.Linear(4 * n_embd, n_embd),  # Project back
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
# ==============================

# ===== TRANSFORMER BLOCK =====
class Block(nn.Module):
    """Transformer block: communication (attention) followed by computation (FFN)."""
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head  # floor(n_embd / n_head)
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)  # Layer normalization
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Apply attention with residual connection
        x = x + self.sa(self.ln1(x))
        # Apply feed-forward with residual connection
        x = x + self.ffwd(self.ln2(x))
        return x
# ============================

# ===== LANGUAGE MODEL =====
class BigramLanguageModel(nn.Module):
    """Simple bigram language model with transformer architecture."""
    
    def __init__(self):
        super().__init__()
        # Token embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # Position embeddings
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Stack of transformer blocks
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)  # Language model head

    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # Get token embeddings
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        # Get position embeddings
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        # Combine token and position embeddings
        x = tok_emb + pos_emb  # (B,T,C)
        # Process through transformer blocks
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            # Reshape for loss calculation
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """Generate new text given a starting sequence."""
        for _ in range(max_new_tokens):
            # Crop context to block_size
            idx_cond = idx[:, -block_size:]
            # Get predictions
            logits, loss = self(idx_cond)
            # Focus on last time step
            logits = logits[:, -1, :]  # (B, C)
            # Convert to probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append to sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx
# =========================

# ===== MODEL INITIALIZATION AND TRAINING =====
# Create model instance
model = BigramLanguageModel()
m = model.to(device)
# Print number of parameters
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    # Evaluate loss periodically
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Get batch of data
    xb, yb = get_batch('train')

    # Forward pass
    logits, loss = model(xb, yb)
    # Backward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate text from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
# ============================================

## Full GPT Model Architecture

### Key Components
1. **Token Embeddings**: Convert input tokens to vectors
2. **Position Embeddings**: Add positional information
3. **Transformer Blocks**: Process information through attention and feed-forward layers
4. **Layer Normalization**: Stabilize training
5. **Language Model Head**: Predict next token probabilities

### Hyperparameters


In [ ]:
# ===== HYPERPARAMETERS =====
batch_size = 16        # Number of independent sequences processed in parallel
block_size = 32        # Maximum context length for predictions
max_iters = 5000       # Total number of training iterations
eval_interval = 100    # How often to evaluate the model
learning_rate = 1e-3   # Step size for gradient descent
eval_iters = 200       # Number of iterations to average over when evaluating
n_embd = 64           # Size of the embedding dimension
n_head = 4            # Number of attention heads
n_layer = 4           # Number of transformer layers
dropout = 0.0         # Probability of dropping out neurons
# ==========================

### Device Selection


In [ ]:
# Device selection: MPS (Apple Silicon) > CUDA > CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon GPU
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU
else:
    device = torch.device("cpu")   # CPU fallback
print(f"Using device: {device}")

### Attention Head Implementation


In [ ]:
class Head(nn.Module):
    """Single head of self-attention."""
    
    def __init__(self, head_size):
        super().__init__()
        # Key, Query, Value projections
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # Create lower triangular mask for causal attention
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        # Compute key, query, and value matrices
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # Compute attention scores
        wei = q @ k.transpose(-2,-1) * C**-0.5  # Scale by sqrt(d_k)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # Apply mask
        wei = F.softmax(wei, dim=-1)  # Convert to probabilities
        wei = self.dropout(wei)
        # Weighted aggregation of values
        v = self.value(x)
        out = wei @ v
        return out

### Multi-Head Attention


In [ ]:
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel."""
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)  # Projection layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Process through each head and concatenate results
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

### Feed-Forward Network


In [ ]:
class FeedFoward(nn.Module):
    """Simple feed-forward network with one hidden layer."""
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # Expand dimension
            nn.ReLU(),                      # Non-linearity
            nn.Linear(4 * n_embd, n_embd),  # Project back
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### Transformer Block


In [ ]:
class Block(nn.Module):
    """Transformer block: communication (attention) followed by computation (FFN)."""
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head  # floor(n_embd / n_head)
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)  # Layer normalization
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Apply attention with residual connection
        x = x + self.sa(self.ln1(x))
        # Apply feed-forward with residual connection
        x = x + self.ffwd(self.ln2(x))
        return x

### Full Language Model


In [ ]:
class BigramLanguageModel(nn.Module):
    """Simple bigram language model with transformer architecture."""
    
    def __init__(self):
        super().__init__()
        # Token embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # Position embeddings
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Stack of transformer blocks
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)  # Language model head

    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # Get token embeddings
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        # Get position embeddings
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        # Combine token and position embeddings
        x = tok_emb + pos_emb  # (B,T,C)
        # Process through transformer blocks
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            # Reshape for loss calculation
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """Generate new text given a starting sequence."""
        for _ in range(max_new_tokens):
            # Crop context to block_size
            idx_cond = idx[:, -block_size:]
            # Get predictions
            logits, loss = self(idx_cond)
            # Focus on last time step
            logits = logits[:, -1, :]  # (B, C)
            # Convert to probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append to sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

### Training Loop


In [ ]:
# Create model instance
model = BigramLanguageModel()
m = model.to(device)
# Print number of parameters
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    # Evaluate loss periodically
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Get batch of data
    xb, yb = get_batch('train')

    # Forward pass
    logits, loss = model(xb, yb)
    # Backward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate text from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

## Key Takeaways

1. **Data Preparation**
   - Tokenization: Convert text to numerical representations
   - Batching: Create efficient training batches
   - Context and Targets: Understand the prediction task

2. **Model Architecture**
   - Token and Position Embeddings
   - Self-Attention Mechanism
   - Transformer Blocks
   - Layer Normalization
   - Language Model Head

3. **Training Process**
   - Loss Function: Cross-Entropy
   - Optimization: AdamW
   - Evaluation: Periodic validation checks
   - Generation: Autoregressive text generation

4. **Key Concepts**
   - Self-Attention: Communication between tokens
   - Positional Encoding: Preserving sequence order
   - Residual Connections: Helping gradient flow
   - Layer Normalization: Stabilizing training
   - Multi-Head Attention: Capturing different relationships
```
